## 1. Input for optional constraints

In [15]:
media_limits = {
    'izervay': {
        'pde': {
            1: {'lower bound': 60, 'upper bound': 100},
            2: {'lower bound': 50, 'upper bound': 116}
        }
    },
    'grizzly': {
        'pde': {
            1: {'lower bound': 0, 'upper bound': 12},
            2: {'lower bound': 150, 'upper bound': 200}
            }
    }
 }

frozen_medias_data = {
    'izervay': {
        'pde': {1: 200.0, 2: 300.0},
        'dtc ctv': {1: 200.0, 2: 200.0}
    },
    'grizzly': {
        'pde': {1: 0.0, 2: 50.0}
    }
}
media_input_percentage = {
    'izervay': {
        'pde': {1: 0.5, 2: 0.2},
        'dtc ctv': {1: 0.3, 2: 0.1}
    },
    'grizzly': {
        'pde': {1: 1, 2: 2}
    }
}
brand_limit = {
    'izervay': {'lower bound': 0, 'upper bound': 1000},
    'grizzly': {'lower bound': 0, 'upper bound': 5000}
}


## 2. Optimization

In [ ]:
from SpendOptimization import SpendOptimization

brand = 'all'

# Instantiate the SpendOptimization class
optimizer = SpendOptimization(
    budget=100,  # Total budget available for allocation
    media_budget_limits=None,  # Optional: Absolute spend limits for each media channel
    media_budget_limits_pct=None,  # Optional: Spend limits as a percentage of prior year spending per media
    locked_media_allocations=None,  # Optional: Pre-allocated spend limits for media channels
    brand=brand,  # Brands to include in the optimization
    brand_budget_constraints=None  # Optional: Budget limits per brand
)

output = optimizer.run()

Optimization terminated successfully    (Exit mode 0)
            Current function value: -28268.094509032195
            Iterations: 185
            Function evaluations: 13433
            Gradient evaluations: 181


## 3. Output

In [18]:
import pandas as pd

records = []
for brand, medias in output['output'].items():
    for media, periods in medias.items():
        for period, metrics in periods.items():
            row = {
                'brand': brand,
                'media': media,
                'period': period
            }
            row.update(metrics)
            records.append(row)

df = pd.DataFrame(records)
df

,brand,media,period,optimal_spending,incremental_dollar
0,izervay,pde,1,0.120998,32.223516
1,izervay,pde,2,0.120998,32.223474
2,izervay,pde,3,0.120997,32.223414
3,izervay,pde,4,0.120998,32.223505
4,izervay,pde,5,0.120998,32.223470
...,...,...,...,...,...
67,xtandi,dtc ctv,8,0.628288,191.222075
68,xtandi,dtc ctv,9,0.628281,191.221349
69,xtandi,dtc ctv,10,0.628283,191.221606
70,xtandi,dtc ctv,11,0.628282,191.221438


In [19]:
df.sum()

brand                 izervayizervayizervayizervayizervayizervayizer...
media                 pdepdepdepdepdepdepdepdepdepdepdepdedtc ctvdtc...
period                                                              468
optimal_spending                                             100.000001
incremental_dollar                                         28268.094509
dtype: object

## Ongoing Components....

In [ ]:
validate_bounds(
    media_limits=media_limits,
    brand_limit=brand_limit,
    frozen_medias_data=frozen_medias_data,
    budget=100000
)

In [ ]:
def validate_bounds(
    file_path=None,
    media_limits=None,
    brand_limit=None,
    budget=1000,
    frozen_medias_data=None,
    media_budget_limits_pct=None,
    
):
    total_media_lower_bound = 0
    total_brand_lower_bound = 0
    total_frozen_spend = 0

    # 1. Validate brand-level bounds
    if brand_limit:
        for brand, bounds in brand_limit.items():
            lb = bounds.get('lower bound', 0)
            ub = bounds.get('upper bound', float('inf'))

            if lb > ub:
                raise ValueError(
                    f"Brand '{brand}': lower bound ({lb}) is greater than upper bound ({ub})."
                )

            total_brand_lower_bound += lb

        if total_brand_lower_bound > budget:
            raise ValueError(
                f"Sum of brand-level lower bounds ({total_brand_lower_bound}) "
                f"exceeds total budget ({budget})."
            )

    # 2. Validate media-level bounds
    if media_limits:
        for brand, media_dict in media_limits.items():
            brand_media_lb = 0

            for media, period_dict in media_dict.items():
                for period, bounds in period_dict.items():
                    lb = bounds.get('lower bound', 0)
                    ub = bounds.get('upper bound', float('inf'))

                    if lb > ub:
                        raise ValueError(
                            f"Brand '{brand}', media '{media}', period {period}: "
                            f"lower bound ({lb}) is greater than upper bound ({ub})."
                        )

                    brand_media_lb += lb

            if brand_limit and brand in brand_limit:
                brand_ub = brand_limit[brand].get('upper bound', float('inf'))
                if brand_media_lb > brand_ub:
                    raise ValueError(
                        f"Sum of media-level lower bounds ({brand_media_lb}) for brand '{brand}' "
                        f"exceeds brand-level upper bound ({brand_ub})."
                    )
                    
            total_media_lower_bound += brand_media_lb

        if total_media_lower_bound > budget:
            raise ValueError(
                f"Total sum of media-level lower bounds across all brands ({total_media_lower_bound}) "
                f"exceeds total budget ({budget})."
            )

    # 3. Validate frozen media spend
    if frozen_medias_data:
        for brand, media_dict in frozen_medias_data.items():
            brand_frozen_total = 0
            for media, period_dict in media_dict.items():
                for period, spend in period_dict.items():
                    total_frozen_spend += spend
                    brand_frozen_total += spend

            if brand_limit and brand in brand_limit:
                brand_bounds = brand_limit[brand]
                lb = brand_bounds.get('lower bound', 0)
                ub = brand_bounds.get('upper bound', float('inf'))

                if not (lb <= brand_frozen_total <= ub):
                    raise ValueError(
                        f"Frozen media spend for brand '{brand}' = {brand_frozen_total} "
                        f"is outside brand-level bounds ({lb}, {ub})."
                    )

        if total_frozen_spend > budget:
            raise ValueError(
                f"Total frozen media spend across all brands ({total_frozen_spend}) "
                f"exceeds total budget ({budget})."
            )

    # 4. Validate media_input_percentage-derived lower bounds
    if media_input_percentage:
        prior_year_df = pd.read_excel(file_path, sheet_name='Media Spending in prior year')
        prior_year_lookup = {
            (row['Brand'].lower(), row['media'].lower(), int(row['period'])): row['spending']
            for _, row in prior_year_df.iterrows()
        }
        
        total_lower_bound = 0
        for brand, media_dict in media_input_percentage.items():
            brand_lower_bound = 0

            for media, period_dict in media_dict.items():
                for period, ratio in period_dict.items():
                    key = (brand.lower(), media.lower(), int(period))
                    prior_spend = prior_year_lookup.get(key, 0)

                    # Get percentage bound from config
                    percent_bound = 0
                    if (
                        brand in media_budget_limits_pct and
                        media in media_budget_limits_pct[brand] and
                        int(period) in media_budget_limits_pct[brand][media]
                    ):
                        percent_bound = media_budget_limits_pct[brand][media][int(period)]

                    # Calculate lower bound as prior × (1 - %)
                    lower = prior_spend * (1 - percent_bound)

                    brand_lower_bound += lower
                    total_lower_bound += lower

            # Check if brand lower bound exceeds brand upper bound
            if brand_limit and brand in brand_limit:
                brand_ub = brand_limit[brand].get('upper bound', float('inf'))
                if brand_lower_bound > brand_ub:
                    raise ValueError(
                        f"Brand '{brand}': sum of media-level lower bounds from media_input_percentage "
                        f"({brand_lower_bound:.2f}) exceeds brand upper bound ({brand_ub})."
                    )

        # Check if total media-level lower bound exceeds total budget
        if total_lower_bound > budget:
            raise ValueError(
                f"Sum of all media-level lower bounds from media_input_percentage ({total_lower_bound:.2f}) "
                f"exceeds total budget ({budget})."
            )
